In [1]:
#import all neccessary libraries
import json
import pandas as pd
from pandas import json_normalize

In [2]:
#open json file

with open('please_work.json', 'r') as json_file:
    data = json.load(json_file)
    
print(data)

In [3]:
#Extract tweet data
tweets = data['data']

#Extract place information
places = {place['id']: place for place in data['includes']['places']}

#Process each tweet
for tweet in tweets:
    tweet_text = tweet['text']
    tweet_id = tweet['id']
    created_at = tweet['created_at']

    #Check if geo information is available
    if 'geo' in tweet:
        place_id = tweet['geo']['place_id']
        
        #Check if place information is available
        if place_id in places:
            place_info = places[place_id]
            country = place_info['country']
            full_name = place_info['full_name']

In [4]:
#add processed tweets into dataframe
df = pd.DataFrame(tweets)

In [5]:
data_list = []

#Process each tweet
for tweet in tweets:
    tweet_text = tweet['text']
    tweet_id = tweet['id']
    created_at = tweet['created_at']

    #Check if geo information is available
    if 'geo' in tweet:
        place_id = tweet['geo']['place_id']
        
        #Check if place information is available
        if place_id in places:
            place_info = places[place_id]
            country = place_info['country']
            full_name = place_info['full_name']

            #Append the information to the list
            data_list.append({
                'text': tweet_text,
                'edit_history_tweet_ids': tweet['edit_history_tweet_ids'],
                'id': tweet_id,
                'created_at': created_at,
                'geo': place_info
            })

#Create a DataFrame from the list
df = pd.DataFrame(data_list)

In [6]:
#expand column to see full information 
pd.set_option('display.max_colwidth', None)

In [7]:
#our 'geo' column needs to be normalized again as the text is still in
#json file format. Normalizing the 'geo' column will create new columns that will need to be
#removed and renamed to our likeness

#Split the '0' column into separate columns
df_geo = json_normalize(df['geo'])
df_geo.columns = [f'geo.{col}' for col in df_geo.columns]

#Concatenate the normalized 'geo' DataFrame with the original DataFrame
df = pd.concat([df, df_geo], axis=1)

#Drop the original 'geo' column
df = df.drop('geo', axis=1)

In [8]:
df = df.drop(columns= ['edit_history_tweet_ids', 'id','geo.geo.type','geo.id','geo.full_name','geo.geo.bbox'])

df = df.rename(columns= {'text':'TEXT', 'created_at':'CREATION DATE',
                         'geo.country': "COUNTRY", 'geo.country_code':'COUNTRY CODE',
                        'geo.name':'STATE'})

#show data
df.tail(1)

,TEXT,CREATION DATE,COUNTRY,COUNTRY CODE,STATE
1,API practice tweet,2023-11-09T01:00:38.000Z,United States,US,New York


**The displayed tweet comes from my own personal twitter account. I consent to it being shown as well its geolocation. As for its geocoordinates, I did drop that column.**

In [12]:
#now export as an excel file (or csv)
with pd.ExcelWriter('LOOK UP TWEETS.xlsx') as writer:
    df.to_excel(writer, sheet_name = "Exported Tweets")